# Importing & data extraction

In [ ]:
import numpy as np
import math
from numpy.linalg import inv
from numpy import linalg as LA
from scipy.special import gamma
import scipy.integrate as integrate
from scipy import integrate
import pandas as pd
#import cupy as cp
from numba import cuda
from random import shuffle
import progressbar

# Theory

## Rule
+ Đơn điệu : cột hay hàng tăng thì lớp không giảm
+ Liên thông (dk cần?): tại hàng (lớp) i thì cột cột đầu phải <i và cột cuối >i 
+ Liên thông (cần và đủ?): ma trận xs chuyển có duy nhất 1 phân phối dừng.
+ Chu kì 1: luôn thỏa mãn khi liên thông và lớp 1 -> lớp 1 khi số vụ = 0

# Formula



$$ p_k = C_k^{k+a-1} (\frac{\tau}{1+\tau})^a(\frac{1}{1+\tau})^k $$

$$ 
  e = \left [\int_0^\infty e_1(\lambda)u(\lambda), ... \int_0^\infty e_s(\lambda)u(\lambda) \right] \tag{2}
$$
Trường hợp $\Lambda$ tuân theo phân phối Gamma
$$
u(\lambda) = \frac{\tau^a e^{-\tau\lambda}\lambda^{a-1}}{\Gamma(a)}
$$
(2.8 tr 196 - 2017)

Trong trường hợp $\Lambda$ tuân theo phân phối IG($\mu,\theta$):
$$u(\lambda)=\sqrt{\frac{\theta}{2\pi \lambda^3}}e^{-\frac{\theta(\lambda-\mu)^2}{2\mu^2\lambda}}$$

$$ b_j = \frac{\int_0^\infty e_j(\lambda) \lambda u(\lambda)d\lambda}{e_j} \tag{4} $$

+ Tính tích phân:
$$Q(b)=\int_0^\infty \sum_{j=1}^s(b_j-\lambda)^2e_j(\lambda)u(\lambda) \ d\lambda    \tag{5}$$

+ Công thức tính $MAE_{\eta}$
$$MAE_{\eta} = \int_0^\infty | 1-\eta(\lambda)|u(\lambda) \ d\lambda$$
Với :
$$\eta(\lambda)=\frac{\partial b_e}{b_e} / \frac{\partial \lambda}{\lambda} = \frac{\partial b_e}{\partial \lambda} / \frac{b_e}{\lambda}$$

# Algorithm

+ Tạo ma trận chuyển lớp B có (n lớp) x (1 đến m vụ): duyệt theo luật \\
Rc

+ Tạo các ma trận luật chuyển $T_k$ từ B (trang 195 - 2017)
$$T_k[i,j] = ( B[i,k] == j )$$ với $ k = \overline{1,n} $

+ Tạo các ma trận xác suất chuyển $P(\lambda)$ từ $T_k$ và $p_k$
$$ P(\lambda) = \sum_{k=0}^n p_k(\lambda) T_k[i,j] $$
với $\lambda = \Delta, 2\Delta, ..., M$
+ Tìm phân phối dừng cho $P(\lambda)$ (nếu có > 1 pp dừng thì loại do ko lt), pp dừng là nghiệm e của hệ
$$ eP = e; \sum_i e_i = 1$$
PPD

+ Tính vector tỉ lệ phí b

+ Tính giá trị hàm mục tiêu Q(b)

+ Lặp lại với các ma trận chuyển và chọn ma trận chuyển có hàm mục tiêu Q(b) nhỏ nhất.




# Parameter ( unfinished )

In [ ]:
# from data
meanX = 0.394
sSquare = 0.5868

# gamma for number of claim
tau_const = 2.042524
a_const = 0.804664
mu=0.05
theta=0.01

### fake

# fake (kind of)

m_dist = 17.650819
s_dist = 3.85621

### fake

b_e_const = a_const/tau_const
variance_const = a_const/(tau_const**2)

# from alg
CRange=np.array([0,5,15,50])

n = 5                                         # số lớp cho khách hàng                                   
m = len(CRange)+1                               # số lớp cho số tiền

Delta = 0.01                             # bước nhảy cho lambda
DeltaTheta = 0.1
#0.0005
M = 2                                         # cận trên cho lambda
MTheta = 100

stepNum = (int) (M/ Delta)
stepNumTheta = (int) (MTheta/ DeltaTheta)     

Lamb = (np.array(range(0, stepNum)) + 10**-4) * Delta       # mảng lambda   

Theta = (np.array(range(0, stepNumTheta))+1) * DeltaTheta

# Function


In [ ]:
#Tạo mảng p_k
def Ckn(k,n):
  return gamma(n+1)/gamma(k+1)/gamma(n-k+1)


# p=[]

# for k in range (1,m+1):
  # p.append(Ckn(k+a_const-1,k)*((tau_const/(1+tau_const)**a_const))*((1/(1+tau_const))**k))

In [ ]:
# Tìm phân phối dừng (chỉ 1 vecto)
def PPD(P):
    n = len(P)
    P = (P - np.eye(n)).T
    P[0] = [1] * n
    b = [1] + [0] * (n-1)
    try:
      ppd = np.linalg.solve(P,b)
      return ppd
    except:
      #print("error, P =")
      #print(P)
      raise ValueError

In [ ]:
#Tạo ma trận chuyển 

def Create_Matrix_T(a):
  # #changed
  n = len(a)
  m = len(a[0])
  # #changed
  
  T=np.zeros((m,n,n))
  for k in range(0,m):
    for i in range(0,n):
      T[k][i][int(a[i][k]-1)]=1
  return T

$p_{c_k, c_{k+1}}(\lambda, \theta)=\mathbb{P}(c_k\leq Z \leq c_{k+1}|\Lambda = \lambda ,\Theta=\theta)=\sum_{i=0}^\infty\mathbb{P}(c_l\leq Z \leq c_{l+1}|N = i,\Theta=\theta)*\mathbb{P}(N = i| \Lambda = \lambda)$

 Ta có:
    $$
    \mathbb{P}(N = i| \Lambda = \lambda) = e^{-\lambda}\frac{\lambda^i}{i!}
    $$
    $$\mathbb{P}(c_l\leq Z \leq c_{l+1}|N = i,\Theta=\theta)=F(c_{l+1})-F(c_l)$$

với F là hàm phân phối của biến ngẫu nhiên Gamma với tham số $i,\frac{1}{\theta}$. $$F(x ; i, \frac{1}{\theta})=1-\sum_{k=0}^{i-1} \frac{(\theta^{-1} x)^{k}}{k!} e^{-\theta^{-1} x}$$

In [ ]:
def giaiThua(n):
    if n == 0:
        return 1
    return n * giaiThua(n - 1)

In [ ]:
def ErlangDis(theta,n,x):
    temp=0
    for k in range(n):
        temp+=(x/theta)**k/giaiThua(k)*math.e**(-x/theta)
    return 1-temp

In [ ]:
def Create_Matrix_P(T, lamb, theta):
  # #changed
  m = len(T)
  n = len(T[0])
  #changed
  P=np.zeros((n,n))
  p =[]
  infty=10
  
  p.append(math.e**(-lamb))
  
  # p.append(math.e**(-lamb) * ErlangDis(theta,i,CRange[1]/2)-ErlangDis(theta,i,CRange[0]))
  for k in range(m-2):
    ap=0
    for i in range(0,infty):
        temp1=math.e**(-lamb)*(lamb)**i/giaiThua(i)
        temp2=ErlangDis(theta,i,CRange[k+1])-ErlangDis(theta,i,CRange[k])
        ap+=temp1*temp2
        # print(ap)
    p.append(ap)
  
  p.append(1-sum(p))

  for k in range(0,m):
    for i in range (0,n):
      for j in range (0,n):
        P[i][j]+=T[k][i][j] * p[k]
  return P

#Gamma and Inverse Gamma


In [ ]:
# Tính tích phân từ hàm dạng mảng
def Integral_Trapezoid(func, stepSize):
  return stepSize*(np.sum(func) - func[0]/2 - func[len(func)-1]/2 )

 $$Gamma=\displaystyle \dfrac{\tau^a e^{-\tau\lambda}\lambda^{a-1}}{\Gamma(a)}$$
 $$Inverse Gamma = \dfrac{\frac{1}{m}e^{-\frac{m}{\theta}}}{{(\frac{\theta}{m})^{s+1}}\Gamma(s)}$$

In [ ]:
def Gamma_uFunc(lamb):
  return tau_const**a_const * math.e**(-tau_const*lamb) * lamb**(a_const-1) / gamma(a_const)

In [ ]:
def IG_uFunc(lamb):
  return math.sqrt(theta/(2*math.pi*(lamb)**3))*math.e**(-theta*(lamb-mu)**2/(2*mu**2*lamb))

In [ ]:
def New_IG_uFunc(theta):
  # if theta < m_dist:
  #   return 0
  return math.e**(-m_dist/theta)*m_dist**s_dist/gamma(s_dist)/theta**(s_dist+1)

<!-- # pareto:
$$
f(x) = \frac{\alpha x^\alpha}
{(x+m)^{\alpha+1}}
$$ -->


#Tích phân 2 lớp Lambda, Theta

In [ ]:
bar = progressbar.ProgressBar(maxval=100, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

U = []  # claim num
V = []  # money                               
for lamb in Lamb:
  #Gamma Distribution:
  U.append(Gamma_uFunc(lamb))

  #IG Distribution:
  #U.append(IG_uFunc(lamb))

for theta in Theta:
  V.append(New_IG_uFunc(theta))

def Objective(B):
  n = len(B)
  m = len(B[0])

  # stepNum = len(Lamb)

  #checked
  print("tìm mảng phân phối dừng\n")
  bar.start()
  
  T = Create_Matrix_T(B)
  # mảng 3 chiều lần lượt theo lambda, theta, thứ tự lớp
  E = []
  for i in range(stepNum):
    smolE = []
    for j in range(stepNumTheta):
    #
      P = Create_Matrix_P(T, Lamb[i], Theta[j])
    #
      try:
        ppd = PPD(P)
      except:
        #print("disconnected")
        return (math.inf, math.inf, math.inf, "disconnected 1")
      smolE.append(ppd)
    E.append(smolE)
    bar.update(i/stepNum * 100)
  bar.finish()

  # tính pp dừng
  print("tìm phân phối dừng hệ thống\n")
  bar.start()

  e = []
  for j in range(n): 
    ek_Integral = []                                                  # số vụ m or n ?
    for i in range(stepNum):    # theo lambda 
      Integral = []
      for k in range(stepNumTheta):
        Integral.append(U[i] * V[k] * E[i][k][j])                                # (2)
      ek_Integral.append(Integral_Trapezoid(Integral, DeltaTheta))

    ej = Integral_Trapezoid(ek_Integral, Delta)

    e.append(ej)
    if ej == 0:
        #print("disconnected")
        #print(np.array(E)[:,j])
        #print(Integral)
      return (math.inf, math.inf, j, "disconnected 2")
    
    bar.update(j/n * 100)
  
  bar.finish()  
  # tính vector phí

  # e = [0.5, 0.3, 0.2]

  b = []
  for j in range(n): 
    bk_Integral = []                                                  # số vụ m or n ?
    for i in range(stepNum):    # theo lambda 
      Integral = []
      for k in range(stepNumTheta):  #theo theta
        Integral.append(U[i]* V[k] * E[i][k][j] * Lamb[i]**4 * Theta[k])                                # (2)
      bk_Integral.append(Integral_Trapezoid(Integral, DeltaTheta))

    bj = Integral_Trapezoid(bk_Integral, Delta)

    b.append(bj/ej)
    if bj == 0:
        #print("disconnected")
        #print(np.array(E)[:,j])
        #print(Integral)
      return (math.inf, math.inf, math.inf, "disconnected 3")

  # /
  # e

  Qb = 0
  Qb_l = []
  for j in range(n):
    Qb_Integral = [] 
    for i in range(stepNum):
      Qbk_Integral = []
      for k in range(stepNumTheta):
        Qbk_Integral.append((b[j] -  Lamb[i] * Theta[k]) **2 * E[i][k][j] * U[i] * V[k])
      Qb_Integral.append(Integral_Trapezoid(Qbk_Integral,DeltaTheta))                        # (5)
    Qb_l.append(Integral_Trapezoid(Qb_Integral, Delta));
  Qb = sum(Qb_l)


  #Gamma Distribution:
  # QN = ( (b*b)@e - (a_const/tau_const)**2 ) / variance_const
  # #IG Distribution:
  # #QN = (b*b@e-mu**2) / (mu**3/ theta)

  b=np.array(b)
  b_e = b@e
  V_b_e = math.sqrt( (b-b_e) * (b - b_e) @ e ) / b_e
  MAE_V = abs(1-V_b_e)

  # RSAL = (b_e-b[0])/(b[n-1]-b[0])

  # #Chuẩn hóa b
  # b=b/b[2]

  return e,b,MAE_V

In [ ]:
Bop = np.array(
         [[1, 2, 2, 3, 4],
         [1, 2, 3, 3, 4],
         [2, 4, 4, 4, 5],
         [3, 4, 4, 5, 5],
         [4, 5, 5, 5, 5]])
e,b,MAE_V = Objective(Bop)
e,b,MAE_V

[                                                                        ] N/A%

tìm mảng phân phối dừng



[                                                                        ] N/A%

tìm phân phối dừng hệ thống



[========================================================================] 100%

([0.7305188581149649,
  0.1658437447014402,
  0.0697190574207058,
  0.07895597664449384,
  0.030647470977425723],
 array([ 4.07114674,  6.22206697,  9.22466717, 26.11462697, 29.6704413 ]),
 0.05722828175576067)

In [ ]:
b_l=np.array([ 4.07114674,  6.22206697,  9.22466717, 26.11462697, 29.6704413 ])
b_l=b_l/b_l[2]
b_l

array([0.44133264, 0.67450314, 1.        , 2.83095601, 3.21642405])

# Main:

In [ ]:
LuuKetqua={'e':[],'b':[],'MAE':[], 'B':[]}

In [ ]:
# LuuKetqua['e'].append(e)
# LuuKetqua['b'].append(b)
# LuuKetqua['Qb'].append(Qb)
# LuuKetqua['B'].append(B)
# LuuKetqua

In [ ]:
#def BM(B):
# B = np.array([
#   [1,4,4,4,4],
#   [1,4,4,4,4],
#   [2,4,4,4,4],
#   [3,4,4,4,4]
# ])

B = np.array( [[1, 2, 2, 4, 4],
   [1, 2, 3, 4, 4],
   [2, 3, 4, 4, 5],
   [3, 4, 4, 5, 5],
   [4, 5, 5, 5, 5]])

# n = len(B)
# m = len(B[0])

bMin = B
current = Objective(B)[2]   # current objective
if current == math.inf:
  raise ValueError

obj = 0
LoopTime=0
MaxLoopTime=200
np.random.seed(44)

List=[]
for i in range(n):
  for j in range(m):
    List.append([i,j])

changed = True
while changed:
  print('Vong lap so '+str(LoopTime)+'\n')
  LoopTime+=1
  if(LoopTime>MaxLoopTime):
     break
  changed = False
  #for row in range(n):
  #  if changed:
  #    break
  #  for col in range(m):
  shuffle(List)
  for x in List:
    if changed:
      break
    row=x[0]
    col=x[1]
    value = B[row][col]
    for element in [value-1, value+1]:
      try:
        if element<=0 or element >= n:
          continue
        B_neighbor = B.copy()
        B_neighbor[row][col] = element

        if (row < n-1 and element > B[row+1][col])\
        or (col < m-1 and element > B[row][col+1])\
        or (row > 0 and element < B[row-1][col])\
        or (col > 0 and element < B[row][col-1]):     
          continue
        
        e_loop, b_loop, obj = Objective(B_neighbor)
        
        if obj < current:
          current = obj
          B = B_neighbor.copy()
          changed = True
          LuuKetqua['e'].append(e_loop)
          LuuKetqua['b'].append(b_loop)
          LuuKetqua['MAE'].append(obj)
          LuuKetqua['B'].append(B)

          continue

      except:
        1

[                                                                        ] N/A%

tìm mảng phân phối dừng



[========================================================================] 100%
[                                                                        ] N/A%

tìm phân phối dừng hệ thống



[                                                                        ] N/A%

...

[==============                                                          ]  20%

tìm phân phối dừng hệ thống



[                                                                        ] N/A%

tìm mảng phân phối dừng



[================================================================        ]  89%

In [ ]:
LuuKetqua

In [ ]:
B

In [ ]:
LuuKetqua2={'E':LuuKetqua['e'],'Qb': LuuKetqua['Qb'], 'b': LuuKetqua['b']}
res=pd.DataFrame(LuuKetqua2)
res.head()
res.to_excel('Version1_MAE.xlsx')

In [ ]:
RES={'B':[]}
for x in LuuKetqua['B']:
    RES['B'].append(x)

In [ ]:
RES

{'B': [array([[1, 2, 2, 4, 4],
         [1, 2, 3, 4, 4],
         [2, 3, 4, 4, 5],
         [3, 4, 4, 5, 5],
         [4, 5, 5, 5, 5]]), array([[1, 2, 2, 4, 4],
         [1, 2, 4, 4, 4],
         [2, 3, 4, 4, 5],
         [3, 4, 4, 5, 5],
         [4, 5, 5, 5, 5]]), array([[1, 2, 2, 4, 4],
         [1, 3, 4, 4, 4],
         [2, 3, 4, 4, 5],
         [3, 4, 4, 5, 5],
         [4, 5, 5, 5, 5]]), array([[1, 2, 2, 4, 4],
         [1, 3, 4, 4, 4],
         [2, 4, 4, 4, 5],
         [3, 4, 4, 5, 5],
         [4, 5, 5, 5, 5]]), array([[1, 2, 2, 4, 4],
         [1, 4, 4, 4, 4],
         [2, 4, 4, 4, 5],
         [3, 4, 4, 5, 5],
         [4, 5, 5, 5, 5]]), array([[1, 2, 3, 4, 4],
         [1, 4, 4, 4, 4],
         [2, 4, 4, 4, 5],
         [3, 4, 4, 5, 5],
         [4, 5, 5, 5, 5]]), array([[1, 2, 4, 4, 4],
         [1, 4, 4, 4, 4],
         [2, 4, 4, 4, 5],
         [3, 4, 4, 5, 5],
         [4, 5, 5, 5, 5]]), array([[1, 3, 4, 4, 4],
         [1, 4, 4, 4, 4],
         [2, 4, 4, 4, 5],
       

In [ ]:
np.savetxt('data.csv', LuuKetqua['B'], delimiter=',')

ValueError: ignored